In [7]:
from keras.models import load_model
from collections import deque
import numpy as np
import pickle
import cv2

In [8]:
model = load_model("/content/drive/My Drive/DeepLearning project/videoClassificationModel/videoClassificationModel")
output_video="/content/drive/My Drive/DeepLearning project/model/output/demoOutput.avi"
with open("/content/drive/My Drive/DeepLearning project/model/videoclassificationbinarizer.pickle", 'rb') as f: lb = pickle.load(f)
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Queue = deque(maxlen=128)

In [ ]:
capture_video = cv2.VideoCapture("/content/drive/My Drive/DeepLearning project/model/tabketenis.mp4")
writer = None
(Height, Width) = (None, None)
Queue = []

while True:
    (taken, frame) = capture_video.read()
    if not taken:
        break
    if Width is None or Height is None:
        (Height, Width) = frame.shape[:2]

    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (244, 224)).astype("float32")
    frame = np.expand_dims(frame, axis=0)
    frame -= mean
    preds = model.predict(frame)[0]
    Queue.append(preds)
    results = np.array(Queue).mean(axis=0)
    i = np.argmax(results)
    label = lb.classes_[i]
    text = "They are playing: {}".format(label)
    cv2.putText(output, text, (45, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (255, 0, 0), 5)

    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")  # Corrected codec
        writer = cv2.VideoWriter("/content/drive/My Drive/DeepLearning project/model/output_videoTabbleTennis.avi", fourcc, 30,
                                 (Width, Height), True)  # Corrected output file path
    writer.write(output)
    # cv2.imshow("In Progress", output)  # Not needed in Colab
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

# Clear the Queue and release resources
Queue.clear()
print("Finalizing")
writer.release()
capture_video.release()

1/1 [==============================] - 0s 236ms/step
